# Visualização dos dados do COVID-19 no Brasil.
> Criação de gráfico interativo utilizando o pacote Altair.

- toc: false 
- badges: true
- comments: true
- categories: [visualiação, covid19, altair]
- image: 

Este post tem o objetivo da mostrar como utilizar o pacote a biblioteca altair, que nos proporciona a criação de gráficos iterativos.

Para realizar a instalação desse pacote, basta digitar no prompt: `pip install altair`. Assim para consultar a documentação onde é encontrado maiores explicações sobre os diferentes tipos de gráficos, no link: https://altair-viz.github.io/.

## Bibliotecas 

Para carregar a biblioteca do altair é só utilizar a `import altair`, afim da simplificação do código iremos chamar esse pacote durante o código como `alt`.

In [30]:
import pandas as pd
import altair as alt

Os dados foi feito o download por meio do site do ministério da saúde e logados no github para acesso aos dados que estão em uma planilha do excel `.xlsx` (https://covid.saude.gov.br/).

In [31]:
df = pd.read_excel('https://github.com/lucastiagooliveira/lucas_repo/blob/master/dados/covid/covid19.xlsx?raw=true')

In [43]:
df.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-25,9,210147125.0,0,0,0,0,NaN,NaN,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-26,9,210147125.0,1,1,0,0,NaN,NaN,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-27,9,210147125.0,1,0,0,0,NaN,NaN,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-28,9,210147125.0,1,0,0,0,NaN,NaN,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-29,9,210147125.0,2,1,0,0,NaN,NaN,NaN


## Plotando o gráfico de óbitos

Podemos plotar o gráfico criando o tipo de seleção que utilizamos o método `selection_single`, para que quando o usuário passar o cursor do mouse seja criada uma ação. Posteriormente, devemos criar os dados e estabelecer o tipo de gráfico e as variáveis utilizada para tal. Após isso deve criar o *layout* do gráfico, que será utilizado o qual pode-se escolher qual o tipo de seleção que aparecerá no gráfico.

Para esse caso será adotado que ao percorrer o gráfico com o mouse irá aparecer a quantidade de óbitos (eixo y) e a data do dado eixo x), que foi estabelecido no método do gráfico `mark_text`.

In [49]:
label = alt.selection_single(
    encodings=['x'], 
    on='mouseover',  
    nearest=True,    
    empty='none'     
)

base = alt.Chart(df.loc[df.regiao == 'Brasil']).mark_bar().encode(
    alt.X('data:T'),
    alt.Y('obitosNovos:Q'),
)

alt.layer(
    base,  
    alt.Chart().mark_rule(color='#aaa').encode(
        x='data:T'
        ).transform_filter(label),
    
    base.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
        ).add_selection(label),
    
   
    base.mark_text(align='right', dx=-15, dy=0).encode(
        text='obitosNovos:Q'
    ).transform_filter(label),
    
    base.mark_text(align='right', dx=-15, dy=15).encode(
        text='data:T'
    ).transform_filter(label),
    
).properties(
    width=800,
    height=400
)


alt.LayerChart(...)

## Gráfico média móvel 

Por conta da imprecisão na data correta das mortes, tem sido divulgado amplamente os gráficos utilizando a média móvel que retrata com mais homogenidade a notificações de mortes semanais.

Para realizar isso será criado um novo dataframe com somente os dados do brasil melhor entendimento dos dados.

In [34]:
new_df = df.loc[df.regiao == 'Brasil']
new_df.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-25,9,210147125.0,0,0,0,0,NaN,NaN,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-26,9,210147125.0,1,1,0,0,NaN,NaN,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-27,9,210147125.0,1,0,0,0,NaN,NaN,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-28,9,210147125.0,1,0,0,0,NaN,NaN,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-29,9,210147125.0,2,1,0,0,NaN,NaN,NaN


Para calcular a média móvel semanal é necessário que seja somada a quantidade de óbitos da última semana e dividir pela quantidade de dias (7 dias). Para armazenar os dados foi criada uma coluna adicinal para armazenar esses dados.

In [40]:
new_df['mediaMovel'] = [round(new_df.obitosNovos[i-7:i].sum()/7,0) if i >= 7 else 0 for i in range(0,len(new_df))]

c:\users\lucas\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [48]:
new_df[['mediaMovel']].tail()

,mediaMovel
194,820.0
195,832.0
196,797.0
197,695.0
198,680.0


Utilizando o mesmo tipo de gráfico apresentado anteriormente, temos o gráfico com as médias móveis.

In [42]:
label = alt.selection_single(
    encodings=['x'],
    on='mouseover',  
    nearest=True,    
    empty='none'     
)

base = alt.Chart(new_df).mark_bar().encode(
    alt.X('data:T'),
    alt.Y('mediaMovel:Q'),
)


alt.layer(
    base,
   
    alt.Chart().mark_rule(color='#aaa').encode(
        x='data:T'
        ).transform_filter(label),
    
    base.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
        ).add_selection(label),

    base.mark_text(align='right', dx=-15, dy=0).encode(
        text='mediaMovel:Q'
    ).transform_filter(label),
    
    base.mark_text(align='right', dx=-15, dy=15).encode(
        text='data:T'
    ).transform_filter(label),
    
).properties(
    width=800,
    height=400
)


alt.LayerChart(...)

Essa é uma amostra da quantidade enorme de recursos que esse pacote possui.